# Read and examine the business data set

For starters I need to get a basic understanding of the data.  It is a **open data set of businesses** in San Diego.  It was obtained from government sources.  I was given an [excel (xlsx) file](https://drive.google.com/file/d/1bsExDRghMkyBrbEuuC1p4VMuna7lBhhH/view).  

- Saved the `data worksheet` from xlsx file as csv (it's just easier)
- Use `pandas` to read and maniputate the dataframe
- First step is understand the data (shape, dtypes, info(), etc)

The `output` from this processing is a new geo dataframe for further, problem specific analysis.

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import osmnx as ox

### Read the data

In [ ]:
biz_df = pd.read_csv('../data/biz_data.csv', sep='\t', index_col=0)

### Examine layout and structure of the dataframe

In [ ]:
biz_df.shape

In [ ]:
biz_df.info()

In [ ]:
biz_df.isnull().sum()

In [ ]:
print("missing business phone numbers: {:.2%}".format(biz_df['BUSINESS PHONE'].isnull().sum() / len(biz_df)))

# Examine, rework, and extend dtypes

Next some basic analysis of dtypes.  There are two types in the data set, int64 and object (str).  The column with the most missing values is the phone number for now that's ok.  The specific values I want to transform are:

1. NAICS - This is an int in the input source. We need a character representation

2. ZIP - this value is really zip+4 as object (i.e. str)
  
3. Dates - There are three date variables.  They are strings in the input data.  It's easy to convert to py datetime so ...

4. Geocode - Using osmnx api to osm

## NAICS

  - Convert to str
  - For starters, use the first to char's for sector (will worry about more detail later)

**TBD: add NAICS reference to explain levels**

In [ ]:
naics_desc = pd.read_excel('../data/2017_NAICS_Descriptions.xls')

naics_desc['Code'] = naics_desc['Code'].astype(str)

lookup_dict = naics_desc[['Code', 'Title']].set_index('Code')['Title'].to_dict()

def lookup(key):
    if key in ('48', '49'):
        key = '48-49'
    if key in ('44', '45'):
        key = '44-45'
    if key in ('31', '32', '33'):
        key = "31-33"
        
    desc = lookup_dict[key]
    if desc[-1:] == 'T':
        desc = desc[:-1]
        
    return desc.rstrip()

In [ ]:
biz_df['sector'] = biz_df['NAICS'].astype(str).apply(lambda x: x[:2])

In [ ]:
biz_df['sector_desc'] = biz_df['sector'].apply(lambda x: lookup(x) + f"({x})")

In [ ]:
biz_df.sector.value_counts()

In [ ]:
biz_df.sector_desc.value_counts(normalize=True) #* 100

### So you see what I did with the transformed data?

  - sector is used for selection
  - sector_desc is used for human consumption (you need to look at the excel referened above)

## ZIP

  - Notice the ZIP column is really ZIP+4
  - Createing new column called zip_code
  - Easy string hack to get the actual zip

In [ ]:
biz_df['zip_code'] = biz_df['ZIP'].apply(lambda x: x.split('-')[0].strip())

In [ ]:
biz_df.iloc[27]

In [ ]:
biz_92110 = biz_df.query(f"zip_code == '92110'").reset_index()

In [ ]:
len(biz_92110)

## ZIP codes around El Cajon Boulevard BID

This section will filter the larger biz_df to include only those in zip codes related to the Blvd BID. <br/>
After a quick look at a map I see two zip codes in the area.<br/>

These query examples us the query method from pandas.  I've noticed some issues running this type of query in collab.

In [ ]:
zips = ['92115', '92116']

In [ ]:
ecb_df = biz_df.query(f"zip_code in @zips")

In [ ]:
len(ecb_df)

In [ ]:
len(ecb_df.query(f"zip_code == '92115'"))

In [ ]:
len(ecb_df.query(f"zip_code == '92116'"))

In [ ]:
print(f"{len(ecb_df) / len(biz_df):.2%} of the businesses are in this area.")

In [ ]:
ecb_df.columns

## Datetime

It appears datetime types not supported in ESRI driver for shape files, so just leave this alone for now.<br/>

For now it's enough to know that this is how you can convert the string to datetime.  <br/>

I guess if one wants to do some geo-temporal analysis it can just be done on the other side in the geodf.

In [ ]:
biz_df.columns

In [ ]:
biz_df['CREATION DT'] = pd.to_datetime(biz_df['CREATION DT'])
biz_df['START DT'] = pd.to_datetime(biz_df['START DT'], errors='coerce')
biz_df['EXP DT'] = pd.to_datetime(biz_df['EXP DT'])

In [ ]:
biz_df.info()

In [ ]:
# build the ecb df again with transformed columns
ecb_df = biz_df.query(f"zip_code in @zips")

In [ ]:
ecb_df.info()

## Gecode

Geocoding uses the Nominatim service of OSM via the osmnx api.  The terms of use state one request per second so the wrapper function handles that.<br/>

If you're executing this code be aware of the timing of things and the size of the dataframe you're trying to geocode. <br/>

I have a few examples below

In [ ]:
def good_address(addr):
    if addr.find('SUITE') > 0:
        return addr[:addr.find('SUITE')-1]
    else:
        return addr

In [ ]:
import time
def geocode_address(row):
    """
    Special function applied to a zip_code transformed row.  
    
    Notes:
      1. nominatim terms of use require one query per sec so we sleep on each iteration.
      2. When we get no match, returning None so we can query for
         invalid geo's later.
    """
    time.sleep(1)
    good_addr = good_address(row.ADDRESS)
    geocode_query = f"{good_addr}, {row.CITY}, {row.STATE}, {row.zip_code}"
    try:
        lat, lon = ox.geocode(geocode_query)
        return Point(lon, lat)
    except:
        print(geocode_query)
        return None #Point(lon, lat).wkt

In [ ]:
# An example for row 27
row = ecb_df.iloc[27]
query = f"{good_address(row.ADDRESS)}, {row.CITY}, {row.STATE}, {row.zip_code}"
print("Address to geocode: " + query)


geocode_address(biz_df.iloc[27]).wkt

### Back-of-the-envelope calculation

In [ ]:
ecb_df.zip_code.value_counts()

In [ ]:
#back of the envelope calc for time to compute - This simple hack estimates the minutes required to geocode.
len(ecb_df) / 60.

**So little over 39 minutes to geocode 92115 and 92116**

**You can stop here as I've previously done this and saved the gdf**

In [ ]:
#%%timeit
ecb_df['geometry'] = ecb_df.apply(lambda r: geocode_address(r), axis=1)

In [ ]:
ecb_gdf = GeoDataFrame(ecb_df, geometry=ecb_df.geometry)

In [ ]:
ecb_gdf.geometry.isnull().sum()

In [ ]:
ecb_gdf.to_file("../data/ecb.shp")

**Read the previously geocoded data**

In [ ]:
previously_computed_gdf = gpd.read_file("../data/ecb.shp")

In [ ]:
denominator = len(previously_computed_gdf)
numerator = previously_computed_gdf['geometry'].isnull().sum()

print(f"Total businesses: {denominator}")
print(f"Total that Nominatim couldn't find: {numerator}")
print(f"Unable to geocode: {numerator/denominator:.2%} addresses")

### That's not bad actually.

**Can anyone suggest an approach to clean the addresses?**

## Final Note on Geocoding

I am currently creating a personal Nominatim server.  I won't have to throttle it so I will geocode and publish the entire file.